In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import glob
from ipywidgets import interact, Dropdown
from alive_progress import alive_bar  # Import the alive-progress bar

# Load the country shapefile
gdf_shp = gpd.read_file("C:\\KIDS\\wb_countries_admin0_10m\\WB_countries_Admin0_10m.shp")

# Folder for GBIF shapefiles
directory = "C:/KIDS/data2024_0321/GBIF_Parallel_GIS_group/demo/"
os.chdir(directory)
extension = '*.shp'

# Find all shapefiles
shp_files = glob.glob(f'{directory}/{extension}')

# Location for output pivot table
output_dir = "C:/KIDS/data2024_0321/species/demo/"

# Process each file
total_data = []

# Start processing each shapefile
for filename in shp_files:
    start_time = time.time()
    file_name = os.path.basename(filename).split('.')[0]
    output_file = f"{output_dir}{file_name}_area_by_country.csv"
    
    if os.path.isfile(filename): 
        gdf = gpd.read_file(filename).dropna(subset=["group"])
        
        # Initialize progress bar for features in the current shapefile
        total_features = len(gdf)
        with alive_bar(total_features, title=f"Processing features in {file_name}", force_tty=True) as bar:
            for index, feature in gdf.iterrows():
                gdf_each = gpd.GeoDataFrame([feature], crs=gdf.crs)
                gdf_each = gdf_each.to_crs(gdf_shp.crs)
                intersection = gpd.overlay(gdf_each, gdf_shp, how='intersection')
                intersection['Area_km2'] = intersection.to_crs('ESRI:54009').geometry.area / 1e6 
                total_data.append(intersection[['species', 'lcat', 'group', 'WB_NAME', 'Area_km2']])
                
                bar()  # Update the progress bar after each feature is processed

        # Save concatenated DataFrame to CSV
        combined_df = pd.concat(total_data, ignore_index=True)
        combined_df.to_csv(output_file)
        print(f"Processed {file_name} in {(time.time() - start_time)/60:.2f} minutes", flush=True)

# Function to create pie charts
def generate_pie_chart(species):
    data = combined_df[combined_df['species'] == species]
    grouped_data = data.groupby('WB_NAME')['Area_km2'].sum()
    grouped_data.plot(kind='pie', autopct='%1.1f%%', ylabel='', title=f'Area Distribution for {species}')
    plt.show()

# Create dropdown for species selection
species_list = combined_df['species'].unique()
dropdown = Dropdown(options=species_list, value='Abantis venosa')

# Display dropdown and bind the pie chart function
interact(generate_pie_chart, species=dropdown)


Processed Group_1_updated in 1.20 minutes


interactive(children=(Dropdown(description='species', options=('Abbottina rivularis', 'Abbreviata antarctica',…

<function __main__.generate_pie_chart(species)>